In [ ]:
# Para el tratamiento de datos:
import pandas as pd
import numpy as np
from datetime import datetime
import re
import unicodedata
import os
pd.set_option('display.max_columns', None) #Para ver todas las columnas
from functools import reduce
import pycountry
from rapidfuzz import process
from iso3166 import countries as iso_countries
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer


### Paso 1: Carga y visualizacion del csv

In [ ]:
df = pd.read_csv("final_powerbi_data/worldbank_data_combined.csv")

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
#veamos los nombres y el numero de los indicadores
print(df["Indicator"].unique())
print(df["Indicator"].nunique())

# Ver cuántos valores hay por indicador
print(df["Indicator"].value_counts())

# Ver datos para un año específico
df[df["Year"] == 2010].head(10)

# Ver datos para un país específico
df[df["Country"] == "Spain"].head(10)



In [ ]:
df[df["Indicator"] == 'Gini index']

### Paso 2: Limpieza

In [ ]:
#Vemos las columnas únicas por indicadores, los años que analizamos y cuantos paises
print("Indicadores únicos:")
print(df["Indicator"].unique())
print("Rango de años:")
print(df["Year"].min(), "a", df["Year"].max())
print("Número de países únicos:", df["Country"].nunique())


In [ ]:
#Los nombres de los indicadores son algo dificultosos, vamos a cambiarlos por algo más legible y simple para que sea más facil leer las gráficas despues en powerBi
nombres_indicadores = {
    'Renewable internal freshwater resources, total (billion cubic meters)': 'Total renewable water (bn m³)',
    'Renewable internal freshwater resources per capita (cubic meters)': 'Renewable water per capita (m³)',
    'Level of water stress: freshwater withdrawal as a proportion of available freshwater resources': 'Water stress (%)',
    'Annual freshwater withdrawals, total (billion cubic meters)': 'Total water withdrawal (bn m³)',
    'Annual freshwater withdrawals, domestic (% of total freshwater withdrawal)': 'Domestic use (%)',
    'Annual freshwater withdrawals, industry (% of total freshwater withdrawal)': 'Industrial use (%)',
    'Annual freshwater withdrawals, agriculture (% of total freshwater withdrawal)': 'Agricultural use (%)',
    'Water productivity, total (constant 2015 US$ GDP per cubic meter of total freshwater withdrawal)': 'Water productivity (GDP/m³)',
    'People using safely managed drinking water services (% of population)': 'Safe drinking water (%)',
    'People using at least basic drinking water services (% of population)': 'Basic drinking water (%)',
    'Average precipitation in depth (mm per year)': 'Average rainfall (mm/year)',
    'Agricultural irrigated land (% of total agricultural land)': 'Irrigated land (%)',
    'Manufacturing, value added (% of GDP)': 'Manufacturing (% GDP)',
    'Industry (including construction), value added (% of GDP)': 'Industry (% GDP)',
    'Gini index': 'Gini index',
    'Poverty headcount ratio at $2.15 a day (2017 PPP) (% of population)': 'Extreme poverty (%)'}

df["Indicator"] = df["Indicator"].replace(nombres_indicadores)

#verificamos el cambio
print(df["Indicator"].nunique())
print(df["Indicator"].unique())



In [ ]:
#sacamos los valores unicos de Country
print(df["Country"].nunique())
print(df["Country"].unique())

In [ ]:
#recuperamos nuestra super funcion de limpiar y normalizar valores
def limpiar_nombres(nombre):
    if pd.isnull(nombre):
        return nombre
    nombre = nombre.strip()
    nombre = re.sub(r'["\'].*?["\']', '', nombre)  # quitamos apodos entre comillas
    nombre = re.sub(r'\(.*?\)', '', nombre)  # quitamos texto entre paréntesis
    nombre = re.sub(r"[-/]", " ", nombre)  #quitamos guiones y barras rarunas
    nombre = " ".join(nombre.split())   # quitamos los multiespacios
    nombre = unicodedata.normalize("NFKD", nombre).encode("ASCII", "ignore").decode("utf-8")  # quitamos acentos y caracteres especiales
    return nombre.title()  # formato título


#aplicamos la funcion a los valores de la columna "Country Name"
df["Country"] = df["Country"].apply(limpiar_nombres)


In [ ]:
# El dataset contiene información tanto de países soberanos como de regiones y agrupaciones económicas.
# Para poder quedarnos con el listado de países soberanos vamos a usar la librería iso3166,
# que nos facilita todos los países y territorios oficialmente definidos por la métrica ISO3166-1 y su código alpha-3 oficial.
# ⚠️ Importante: la ISO no diferencia entre país soberano y territorio dependiente. Ver después para el análisis.

# Primero sacamos los países oficiales y no incluir las agrupaciones
paises_soberanos = {c.name for c in iso_countries}

# Creamos la función usando fuzzy matching que nos ayuda a limpiar nombres de países escritos de forma distinta
def normalizar_a_soberano(nombre):
    if pd.isnull(nombre):
        return None
    match, score, _ = process.extractOne(nombre, paises_soberanos)
    return match if score > 95 else None

# Aplicamos la función para normalizar nombres
df["Country"] = df["Country"].apply(normalizar_a_soberano)

# Filtramos los nulos resultantes
df = df[df["Country"].notnull()]

# Al listado de países con los que nos quedamos le pasamos el código alpha-3 para evitar duplicaciones
diccionario_name_to_code = {c.name: c.alpha3 for c in iso_countries}

#Usamos .loc para evitar SettingWithCopyWarning
df.loc[:, "Country Code"] = df["Country"].map(diccionario_name_to_code)

# Eliminamos duplicados por combinación única de país, indicador y año
df = df.drop_duplicates(subset=["Country", "Indicator", "Year"], keep="first")


In [ ]:
#sacamos los valores unicos de Country
print(df["Country"].nunique())
print(df["Country"].unique())

In [ ]:
#vemos que aun pasando la iso nos quedan algunas zonas que excluimos manualmente
excluir_manual = [
    'American Samoa',
    'Aruba',
    'Bermuda',
    'Cayman Islands',
    'Faroe Islands',
    'French Polynesia',
    'Gibraltar',
    'Greenland',
    'Guam',
    'New Caledonia',
    'Northern Mariana Islands',
    'Puerto Rico',
    'Turks and Caicos Islands']

df = df[~df["Country"].isin(excluir_manual)]
print(f"Países únicos tras el filtrado: {df['Country'].nunique()}")
print(f"Países únicos tras el filtrado: {df['Country'].unique()}")

In [ ]:
#verificamos si hay duplicados completos y si hay duplicados con nuestra super combinacion mágica unica
print(df.duplicated().sum())
df.duplicated(subset=["Country", "Indicator", "Year"]).sum()


In [ ]:
#Para facilitar la lectura de los gráficos en PowerBi, vamos a agrupar los paises en macro regiones.
#asi podremos ver tanto los datos de cada pais como los de su región y sub-región segun escogamos el filtro.

#Cargamos el listado de la ONU
df_m49 = pd.read_csv("raw_worldbank_data/ONU-M49.csv")

df_m49.info()

In [ ]:
# Seleccionamos las columnas necesarias
df_m49_simple = df_m49[["ISO3166-1-Alpha-3", "Region Name", "Sub-region Name"]]

# Hacemos el merge usando Country Code que es ISO alpha-3 del 
# nuevo dataset y nepues eliminanos las culumnas no necesarias
df = df.merge(df_m49_simple, left_on="Country Code", right_on="ISO3166-1-Alpha-3", how="left")
df.drop(columns=["ISO3166-1-Alpha-3"], inplace=True)

In [ ]:
#ORdenamos las columnas para que tenga más sentido
columnas_ordenadas = ['Country', 'Country Code', 'Region Name', 'Sub-region Name', 'Year', 'Indicator', 'Indicator Code', 'Category', 'Value']
df = df[columnas_ordenadas]


In [ ]:
df.info()

In [ ]:
#Comprobamos los valores unicos
print("Valores únicos de 'Region Name':")
print(df["Region Name"].nunique())
print(df["Region Name"].unique())
print("\nValores únicos de 'Sub-region Name':")
print(df["Sub-region Name"].nunique())
print(df["Sub-region Name"].unique())

In [ ]:
#vemos las estadisticas basicas
df.groupby("Indicator")["Value"].describe().round(2)


In [ ]:
#water stress deberia estar entre 0 y 10, debe haberse producido un error de unidad, limitamos el valor a 100
df.loc[(df["Indicator"] == "Water stress (%)") & (df["Value"] > 100), "Value"] = 100


In [ ]:
### PASO 3: Gestion de nulos

In [ ]:
#vemos los datos nulos
total = len(df)
nulos_df = pd.DataFrame({
    'Nulos': df.isnull().sum(),
    'Porcentaje': (df.isnull().sum() / total * 100).round(2)}).sort_values(by='Nulos', ascending=False)
print("Nulos y porcentaje por columna:")
print(nulos_df)

In [ ]:
paises_nan_region = df[df["Region Name"].isnull()]["Country"].unique()
print("Países sin región asignada:")
print(paises_nan_region)


In [ ]:
#Kosovo no tiene un reconocimiento unanimo como pais soberano por parte de otros paises, por lo que no tiene region en los listados internacionales como la M49.
#Lo imputamos manualmente y comprobamos.
df.loc[df["Country"] == "Kosovo", ["Region Name", "Sub-region Name"]] = ["Europe", "Southern Europe"]
print(df[["Region Name", "Sub-region Name"]].isnull().sum())



In [ ]:
#para ubicar donde estan los valores nulos vamos a sacar el numero de nulos por indicador y por pais.
nulos_indicador = df[df["Value"].isnull()].groupby("Indicator")["Country"].nunique().sort_values(ascending=False)
print("numero países con nulos por indicador:", nulos_indicador)

nulos_pais = df[df["Value"].isnull()].groupby("Country")["Indicator"].nunique().sort_values(ascending=False)
print("\nnumero indicadores con nulos por pais:", nulos_pais)


In [ ]:
#tambien vemos que hay 3 indicadores (Irrigated land, Extreme poverty y Gini index) de los que casi la totalidad de los paises no tienen datos, por lo que tambien pasamos a eliminarlos.
indicadores_a_eliminar = ["Irrigated land (%)", "Extreme poverty (%)", "Gini index"]
df = df[~df["Indicator"].isin(indicadores_a_eliminar)]


In [ ]:
#vemos que hay paises que no tienendatos en ninguno de los indicadores, por lo que no nos sirven para el analisis y los eliminamos
paises_todos_nulos = df[df["Value"].isnull()].groupby("Country")["Indicator"].nunique()
paises_sin_datos = paises_todos_nulos[paises_todos_nulos == df["Indicator"].nunique()].index
df = df[~df["Country"].isin(paises_sin_datos)]

In [ ]:
#vemos de nuevo los datos nulos
total = len(df)
nulos_df = pd.DataFrame({
    'Nulos': df.isnull().sum(),
    'Porcentaje': (df.isnull().sum() / total * 100).round(2)}).sort_values(by='Nulos', ascending=False)
print("Nulos y porcentaje por columna:")
print(nulos_df)

In [ ]:
df.info()

In [ ]:
df.sample(10)

tenemos un 11% de nulos que viene directamente desde la fuente, que es el Banco Mundial la que nos da los datos oficiales. La ausencia de iertos datos no se trata de un error, si no que nos da a entender que algo paso en ese momento para que no se pudieran obtener los datos: conflictos armados, crisis institucionales, falta de recursos, cambio de regimen y censura,...........
Imputar esos datos pueden llevarnos a maquillar el contexto geopolitico. Se abre el debate de imputar los nulos o no?
Imputar esos datos en estos contextos puede resultar engañoso o poco ético pues la falta de esos valores ya en si es informacion.
Por contrario si queremos ver las tendencias y facilitar los gráficos en powerbi, creo que se debe imputar con tecnicas avanzadas.

In [ ]:
#guardamos el dataframe limpio
#os.makedirs("final_powerbi_data", exist_ok=True)
#output_path = "final_powerbi_data/clean_final_data.csv"
#df.to_csv(output_path, index=False)
#print(f"Archivo guardado correctamente en: {output_path}")

Frente a este conflicto ético, hemos guardado un CSV limpio y sin imputar, con una gestion de los datos estricta. Dado que nuestro interes mayoritario es hacer gráficas y ver las tendencias y patrones decicimos utilizar un metodo de imputacion avazanda para completar ese porcentaje de nulos.

In [ ]:
df.groupby("Indicator")["Value"].describe().round(2)

In [ ]:

# 1. Pivoteamos a formato ancho (indicadores como columnas)
df_wide = df.pivot_table(index=["Country", "Year"], columns="Indicator", values="Value")

# 2. Guardamos medias y desviaciones para desescalar después
means = df_wide.mean()
stds = df_wide.std()

# 3. Escalamos para que KNN funcione correctamente
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_wide)

# 4. Aplicamos imputación con KNN
imputer = KNNImputer(n_neighbors=10)
df_imputed_scaled = imputer.fit_transform(df_scaled)

# 5. Desescalamos manualmente para volver a la escala original
df_imputed = pd.DataFrame(df_imputed_scaled, index=df_wide.index, columns=df_wide.columns)
df_imputed = df_imputed * stds + means

# 6. Restablecemos índice para convertir Country y Year en columnas
df_imputed = df_imputed.reset_index()

# 7. Añadimos columnas auxiliares: código de país, región, subregión
columnas_auxiliares = [
    "Country", "Country Code", "Region Name", "Sub-region Name"
]
df_aux = df[columnas_auxiliares].drop_duplicates()

# 8. Hacemos el merge
df_imputed = df_imputed.merge(df_aux, on="Country", how="left")

# 9. Reordenamos columnas para dejarlo bonito
columnas_indicadores = [col for col in df_imputed.columns if col not in columnas_auxiliares + ["Year"]]
df_imputed = df_imputed[["Country", "Country Code", "Region Name", "Sub-region Name", "Year"] + columnas_indicadores]



In [ ]:
df_imputed.info()

In [ ]:
df_imputed.describe().round(2)

In [ ]:
#arreglamos los pequeños desajustes que se han producido tras la imputacion
vars_no_negativos = [
    "Agricultural use (%)", "Domestic use (%)", "Industrial use (%)",
    "Renewable water per capita (m³)", "Total renewable water (bn m³)",
    "Total water withdrawal (bn m³)", "Water productivity (GDP/m³)"]
for col in vars_no_negativos:
    df_imputed.loc[df_imputed[col] < 0, col] = 0

# Recortar al 100% los indicadores que no pueden pasar ese límite
vars_max_100 = [
    "Water stress (%)", "Basic drinking water (%)", "Safe drinking water (%)",
    "Agricultural use (%)", "Industrial use (%)", "Domestic use (%)"
]
for col in vars_max_100:
    df_imputed.loc[df_imputed[col] > 100, col] = 100


In [ ]:
df_imputed.describe().round(2)

In [ ]:
#guardamos el archivo sin nulos
os.makedirs("final_powerbi_data", exist_ok=True)
output_path = "final_powerbi_data/water_data_wide.csv"
df_imputed.to_csv(output_path, index=False)
print(f"Archivo final guardado en: {output_path}")